## Objective

The goal of this notebook is to explore the features of the NCR patient data and identify those that appear to be the best predictors of survival outcomes.

First, we load the patient data from the 'knownPalliativeTreatments' view, which excludes patients who weren't given a known treatment. 
Before interpretation, we drop rows that are missing progression information or important covariates.

In [ ]:
import pandas as pd 
import pymysql

def load_data(duration_col, event_col):
    db_connection = pymysql.connect(
        read_default_file='/home/jupyter/.my.cnf',
        read_default_group='RAnalysis', 
        db = 'actin_personalization_v2'
    )

    query = "SELECT * FROM palliativeReference"

    df = pd.read_sql(query, db_connection)
    db_connection.close()
    
    return df.dropna(subset = [duration_col, event_col]).copy()

## Numeric interpretation of features

Many categorical features present in the patient data represent ordinal, interval, or ratio quantities and would convey more information to regression techniques in a numerical representation. Here, we map the string values for these features to numbers that represent their meaning:

In [ ]:
from numpy import nan

stageTnm_lookup = {
    "ZERO": 0.0,
    "I": 1.0,
    "IA1": 1.0,
    "IA": 1.0,
    "IA2": 1.0,
    "IA3": 1.0,
    "IB": 1.0,
    "II": 2.0,
    "IIA": 2.0,
    "IIB": 2.0,
    "IIC": 2.0,
    "III": 3.0,
    "IIIA": 3.0,
    "IIIB": 3.0,
    "IIIC": 3.0,
    "IV": 4.0,
    "IVA": 4.0,
    "IVB": 4.0,
    "IVC": 4.0,
    "M": 4.0,
    "NA": nan,
    "X": nan,
}

tnmM_lookup = {
    "M0": 0,
    "M1": 1,
    "M1A": 1.1,
    "M1B": 1.2,
    "M1C": 1.3,
    "M_MINUS": 0,
    "X": nan,
}

tnmN_lookup = {
    "N0": 0,
    "N1": 1,
    "N1A": 1.1,
    "N1B": 1.2,
    "N1C": 1.3,
    "N1M": 1,
    "N2": 2,
    "N2A": 2.1,
    "N2B": 2.2,
    "X": nan
}

tnmT_lookup = {
    "T0": 0,
    "T_IS": 0.5,
    "T1": 1,
    "T2": 2,
    "T3": 3,
    "T4A": 4.1,
    "T4B": 4.2,
    "X": nan
}

lookup_dictionary = {
    "anorectalVergeDistanceCategory": {
        "ZERO_TO_FIVE_CM": 2.5,
        "FIVE_TO_TEN_CM": 7.5,
        "TEN_TO_FIFTEEN_CM": 12.5,
        "OVER_FIFTEEN_CM": 17.5,
    },
    "numberOfLiverMetastases": {
        "ONE": 1,
        "TWO": 2,
        "THREE": 3,
        "FOUR": 4,
        "FIVE_OR_MORE": 5,
        "MULTIPLE_BUT_EXACT_NUMBER_UNKNOWN": 5  # The median number of multiple metastases is 5+
    },
    "asaAssessmentAtMetastaticDiagnosis": {
        "I": 1,
        "II": 2,
        "III": 3,
        "IV": 4,
        "V": 5,
        "VI": 6,
    },
    "venousInvasionDescription": {  # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1769571/
        "EXTRAMURAL": 1.2,
        "INTRAMURAL": 1,
        "NA": nan,
        "NONE": 0,
        "SUSPECT": 0.7,
    },
    "lymphaticInvasionCategory": {
        "NONE": 0,
        "PRESENT": 1,
        "SUSPECT": 0.7,
        "NA": nan
    },
    "extraMuralInvasionCategory": {
        "NA": 0,
        "LESS_THAN_FIVE_MM": 3,
        "ABOVE_FIVE_MM": 7,
    },
    "tumorRegression": {
        "CANNOT_BE_DETERMINED": nan,
        "FULL_REGRESSION": 1,
        "MINIMAL_FOCI": 0.8,  # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4946373/
        "MINIMAL_REGRESSION": 0.2,
        "MODERATE_REGRESSION": 0.5,
        "NO_SIGNS_OF_REGRESSION": 0,
        "NA": nan,
    },
    "differentiationGrade": {
        "GRADE_1_OR_WELL_DIFFERENTIATED": 1,
        "GRADE_2_OR_MODERATELY_DIFFERENTIATED": 2,
        "GRADE_3_OR_POORLY_DIFFERENTIATED": 3,
        "GRADE_4_OR_UNDIFFERENTIATED_OR_ANAPLASTIC_OR_GGG4": 4
    },
    "pathologicalTnmT": tnmT_lookup,
    "pathologicalTnmN": tnmN_lookup,
    "pathologicalTnmM": tnmM_lookup,
    "clinicalTnmT": tnmT_lookup,
    "clinicalTnmN": tnmN_lookup,
    "clinicalTnmM": tnmM_lookup,
    "clinicalTumorStage": stageTnm_lookup,
    "pathologicalTumorStage": stageTnm_lookup
}

def numerize(df):
    for column, lookup in lookup_dictionary.items():
        df[column] = df[column].apply(lookup.get)
    return df

## Standardization

Transform numeric features to have mean 0 and standard deviation 1 in order to compare between features in standard deviation units:

In [ ]:
from sklearn.preprocessing import StandardScaler

def standardize(df, features):
    scaler = StandardScaler()
    
    cols_to_standardize = [f for f in features if pd.api.types.is_numeric_dtype(df[f])]
    df.loc[:, cols_to_standardize] = scaler.fit_transform(df.loc[:, cols_to_standardize])
    return df

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

def imputeWho(df, k):
    columns = ['whoAssessmentAtMetastaticDiagnosis']
    imputer = KNNImputer(n_neighbors=k)
    df[columns] = imputer.fit_transform(df[columns])
    scaler = StandardScaler()
    df.loc[:, columns] = scaler.fit_transform(df.loc[:, columns])
    return df

## Cox proportional hazard model generation

For a given formula, build a Cox model stratified by systemic treatment plan. [Stratification is recommended for a feature that does not obey the proportional hazards assumption](https://lifelines.readthedocs.io/en/stable/Survival%20Regression.html#stratification).

Since we aim to prove that treatment effectiveness can be predicted by some combination of patient features, we'd be doing ourselves a disservice if we assumed proportional hazards in a group with variable treatments. As stated in [the 'lifelines' documentation](https://lifelines.readthedocs.io/en/stable/jupyter_notebooks/Proportional%20hazard%20assumption.html):

> The proportional hazard assumption is that all individuals have the same hazard function, but a unique scaling factor infront.

Put another way, if the proportional hazard assumption holds and the survival curves never cross, our recommendation should be to give all patients the same treatment, i.e. the one whose curve is consistently at the top. Instead, we stratify by treatment: 

In [ ]:
from lifelines import CoxPHFitter

def cox_model(df, formula, duration_col, event_col):
    return CoxPHFitter().fit(
        df,
        duration_col=duration_col,
        event_col=event_col,
        formula=formula,
        strata=["firstSystemicTreatmentAfterMetastaticDiagnosis"]
    )

A 'dataclass' is used for each test result for easy conversion from list of objects to 'DataFrame':

In [ ]:
from dataclasses import dataclass
from math import log10

@dataclass
class TestResult:
    feature: str
    survival_impact: float
    p_val: float
    dof_delta: int
    log_likelihood_ratio: float


## Testing the features

### Model generation
To evaluate the predictive value of each feature, we filter our base data frame down to the rows where that feature is provided. We then generate a reduced Cox model from the filtered data, using just our standard covariates ('ageAtDiagnosis + whoAssessmentAtMetastaticDiagnosis + diagnosisYear') as well as a full model that also considers the feature in question.

### Model evaluations
To see if the feature adds significant predictive value, we compute the [log likelihood ratio](https://en.wikipedia.org/wiki/Likelihood-ratio_test) between the provided log likelihoods for the full and reduced models. This is used along with the change in degrees of freedom (1 for a numeric feature or n-1 for a categorical feature with n categories) in a [chi-squared test](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test) to determine 'p'. The results for all features are collected in a list, which is then converted to a 'DataFrame'. The negative log of 'p' is added in a separate column for use in creating volcano plots:

In [ ]:
from math import log10
from scipy.stats.distributions import chi2

def likelihood_ratio(llmin, llmax):
    return 2 * (llmax - llmin)

def test_feature(df, simple_formula, feature, duration_col, event_col):
    try:
        df_clean = df.dropna(subset = [feature])
        reduced_model = cox_model(df_clean, simple_formula, duration_col, event_col)
        full_model = cox_model(df_clean, " + ".join([feature, simple_formula]), duration_col, event_col)
        dof = 1 if pd.api.types.is_numeric_dtype(df[feature]) else df[feature].nunique() - 1
        log_likelihood_ratio = likelihood_ratio(reduced_model.log_likelihood_, full_model.log_likelihood_)
        p = chi2.sf(log_likelihood_ratio, dof)
        print(f"Testing feature {feature}: p-value {p}, exp(coeff) {full_model.hazard_ratios_[0]}")
        return TestResult(feature, full_model.hazard_ratios_[0], p, dof, log_likelihood_ratio)
    except Exception as e:
        print(f"Failed to test {feature}: {str(e)}")
        return None
    
def test_features(df, base_formula, features, duration_col, event_col):
    result = [r for r in (test_feature(df, base_formula, f, duration_col, event_col) for f in features) if r is not None]
    
    result_df = pd.DataFrame(result)
    result_df["log10_p"] = result_df["p_val"].apply(lambda p: -log10(p))
    return result_df

### Volcano plot

The volcano plot shows which features could be most useful for prediction purposes. The "survival impact" shown on the x-axis is the coefficient estimated by the Cox model produced for that feature. Features with survival impacts greater than 1 imply a positive effect on survival, while those less than one imply poorer survival outcomes. Higher placement on the y-axis ('-log(p)') indicates a lower p-value and thus higher statistical significance.

In [ ]:
import seaborn

def volcano_plot(result_df):
    seaborn.scatterplot(result_df[result_df["p_val"] < 0.1], x="survival_impact", y="log10_p")

## Correlation analysis

We compute the pairwise correlation between all features, and assume no correlation for features where this cannot be evaluated. Next, we hierarchically cluster the correlations and reorder them to show the most correlated features together (one feature, 'tnmPM', is manually reordered to put it beside other TNM features with which it is highly correlated). We create a heatmap from the clustered correlations and see that most features are uncorrelated.

In [ ]:
from numpy import argsort
import scipy.cluster.hierarchy as sch

def cluster_corr(corr_df):
    pairwise_distances = sch.distance.pdist(corr_df)
    linkage = sch.linkage(pairwise_distances, method='complete')
    cluster_distance_threshold = pairwise_distances.max()/2
    idx_to_cluster_array = sch.fcluster(linkage, cluster_distance_threshold, criterion='distance')
    idx = argsort(idx_to_cluster_array)
    
    return corr_df.copy().iloc[idx, :].T.iloc[idx, :]

def plot_pairwise_correlation(df, features):
    correlation = df[features].corr('pearson')
    clustered_corr = cluster_corr(correlation.fillna(0))
    # idx = list(range(0, 3)) + [25] + list(range(4, 24)) + list(range(26, len(clustered_corr)))
    # corrected_corr = clustered_corr.copy().iloc[idx, :].T.iloc[idx, :]
    heatmap = seaborn.heatmap(clustered_corr, xticklabels=True, yticklabels=True)
    heatmap.figure.set_figwidth(12)
    heatmap.figure.set_figheight(8.5)
    

## Feature selection

We essentially want to consider all features that aren't already used in initial filtering or directly related to survival outcomes (e.g. response):

In [ ]:
features = [
    'sex',
    'ageAtDiagnosis',
    'ageAtMetastaticDiagnosis',
    'numberOfPriorTumors',
    'hasDoublePrimaryTumor',
    'primaryTumorType',
    'primaryTumorLocation',
    'sidedness',
    'anorectalVergeDistanceCategory',
    'mesorectalFasciaIsClear',
    'distanceToMesorectalFasciaMm',
    'differentiationGrade',
    'clinicalTnmT',
    'clinicalTnmN',
    'clinicalTnmM',
    'pathologicalTnmT',
    'pathologicalTnmN',
    'pathologicalTnmM',
    'clinicalTumorStage',
    'pathologicalTumorStage',
    'investigatedLymphNodesCountPrimaryDiagnosis',
    'positiveLymphNodesCountPrimaryDiagnosis',
    'presentedWithIleus',
    'presentedWithPerforation',
    'venousInvasionDescription',
    'lymphaticInvasionCategory',
    'extraMuralInvasionCategory',
    'tumorRegression',

    'daysBetweenPrimaryAndMetastaticDiagnosis',
    'hasLiverOrIntrahepaticBileDuctMetastases',
    'numberOfLiverMetastases',
    'maximumSizeOfLiverMetastasisMm',
    'hasLymphNodeMetastases',
    'investigatedLymphNodesCountMetastaticDiagnosis',
    'positiveLymphNodesCountMetastaticDiagnosis',
    'hasPeritonealMetastases',
    'hasBronchusOrLungMetastases',
    'hasBrainMetastases',
    'hasOtherMetastases',        

    'whoAssessmentAtMetastaticDiagnosis',
    'asaAssessmentAtMetastaticDiagnosis',
    'lactateDehydrogenaseAtMetastaticDiagnosis',
    'alkalinePhosphataseAtMetastaticDiagnosis',
    'leukocytesAbsoluteAtMetastaticDiagnosis',
    'carcinoembryonicAntigenAtMetastaticDiagnosis',
    'albumineAtMetastaticDiagnosis',
    'neutrophilsAbsoluteAtMetastaticDiagnosis',

    'hasHadPrimarySurgeryPriorToMetastaticDiagnosis',
    'hasHadPrimarySurgeryAfterMetastaticDiagnosis',
    'hasHadGastroenterologySurgeryPriorToMetastaticDiagnosis',
    'hasHadGastroenterologySurgeryAfterMetastaticDiagnosis',
    'hasHadHipecPriorToMetastaticDiagnosis',
    'hasHadHipecAfterMetastaticDiagnosis',
    'hasHadPrimaryRadiotherapyPriorToMetastaticDiagnosis',
    'hasHadPrimaryRadiotherapyAfterMetastaticDiagnosis',

    'hasHadMetastaticSurgery',
    'hasHadMetastaticRadiotherapy',
    'charlsonComorbidityIndex',
    'hasAids',
    'hasCongestiveHeartFailure',
    'hasCollagenosis',
    'hasCopd',
    'hasCerebrovascularDisease',
    'hasDementia',
    'hasDiabetesMellitus',
    'hasDiabetesMellitusWithEndOrganDamage',
    'hasOtherMalignancy',
    'hasOtherMetastaticSolidTumor',
    'hasMyocardialInfarct',
    'hasMildLiverDisease',
    'hasHemiplegiaOrParaplegia',
    'hasPeripheralVascularDisease',
    'hasRenalDisease',
    'hasLiverDisease',
    'hasUlcerDisease',

    'hasMsi',
    'hasBrafMutation',
    'hasBrafV600EMutation',
    'hasRasMutation',
    'hasKrasG12CMutation',
]

In [ ]:
confidence_level = 0.05

def _find_first_insignificant_index(sorted_df, test_count):
    for index, value in enumerate(sorted_df['p_val']):
        if (value > (index + 1) * confidence_level / test_count):
            return index
            
def benjamini_hochberg(df, test_count):
    sorted_df = df.sort_values(by='p_val')
    return sorted_df.iloc[:_find_first_insignificant_index(sorted_df, test_count)]

In [ ]:
duration_col = "daysBetweenTreatmentStartAndProgression"
event_col = "hadProgressionEvent"
base_formula = "ageAtMetastaticDiagnosis + whoAssessmentAtMetastaticDiagnosis + diagnosisYear"

df = standardize(numerize(load_data(duration_col, event_col)), features)
k = 105
df = imputeWho(df, k)
result_df = test_features(df, base_formula, features, duration_col, event_col)

In [ ]:
duration_col = "survivalDaysSinceMetastaticDiagnosis"
event_col = "hadSurvivalEvent"
base_formula = "ageAtMetastaticDiagnosis + whoAssessmentAtMetastaticDiagnosis + diagnosisYear"

os_df = standardize(numerize(load_data(duration_col, event_col)), features)
k = 105
os_df = imputeWho(os_df, k)
os_results = test_features(os_df, base_formula, features, duration_col, event_col)

In [ ]:
benjamini_hochberg(os_results, len(features))

In [ ]:
pfs_df = df
pfs_results = result_df

In [ ]:
benjamini_hochberg(pfs_results, len(features))

In [ ]:
volcano_plot(result_df)

In [ ]:
result_df[result_df["p_val"] < 0.05]

In [ ]:
plot_pairwise_correlation(df, features)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

# Replace 4 with 3 since there are only 9 of these:
who_df = df.dropna(subset=["whoAssessmentAtMetastaticDiagnosis"]).replace({"whoAssessmentAtMetastaticDiagnosis": {4: 3}})

for col_name in who_df.columns:
    if who_df[col_name].dtype == 'object':
        who_df[col_name] = who_df[col_name].astype('category').cat.codes

x = who_df[features + ['ageAtMetastaticDiagnosis', 'diagnosisYear']].astype(pd.SparseDtype("float", np.nan))
y = who_df["whoAssessmentAtMetastaticDiagnosis"].astype(float)  # Keep as float for regression

k_max = 50
error_rates = []

def error_rate(x, y, k):
    knn = KNeighborsRegressor(n_neighbors=k)
    score = cross_val_score(knn, x, y, cv=10, scoring='neg_mean_squared_error')
    rate = -score.mean()  # Convert negative MSE to positive MSE for error rates
    print(str(k) + ": " + str(rate))
    return rate

for n in range(1, 50):
    error_rates.append(error_rate(x, y, n))

# k = 7

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(1, 50), error_rates)
plt.xlabel("K")
plt.ylabel("Error Rate")